# When Papa Atom Met Mama Atom
> A test of Bokeh and Altair in Fastpages.

- toc: true 
- badges: true
- comments: true
- categories: [altair, bokeh, elements]
- image: images/periodic-table-bokeh.png

## Bokeh

Let's use this example from [Bokeh documentation](https://docs.bokeh.org/en/latest/docs/gallery/periodic.html) to display an interactive Periodic Table of Elements.

A standard Bokeh workflow for displaying plots inline in Jupyter Notebook or Jupyter Lab goes like this:

1) call `output_notebook()`,  
2) construct our plot object `p`,  
3) pass the plot object into `show()`.  

The plots show up in the notebook but do not survive the process of conversion to a static HTML page.  What do we do?

Using `bokeh.embed.components`, we can extract the JS and HTML snippets.  Then we recombine them into one JS+HTML chunk with f-string formatting, and render the plot with the help of `IPython.display`.

In [1]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap

In [2]:
#collapse-show

periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

cmap = {
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "metal"                : "#d93b43",
    "halogen"              : "#999d9a",
    "metalloid"            : "#e08d49",
    "noble gas"            : "#eaeaea",
    "nonmetal"             : "#f1d4Af",
    "transition metal"     : "#599d7A",
}

TOOLTIPS = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass} Da"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Melting point", "@{melting point} K"),
    ("Boiling point", "@{boiling point} K"),
    ("Density", "@{density} g/mL"),
    ("Ionic radius", "@{ion radius} pm"),
    ("Electronegativity", "@{electronegativity}"),
    ("Year Discovered", "@{year discovered}"),
]

p = figure(title="Periodic Table (omitting LA and AC Series)", width=1000, height=450,
           x_range=groups, y_range=list(reversed(periods)),
           tools="hover", toolbar_location=None, tooltips=TOOLTIPS)

r = p.rect("group", "period", 0.95, 0.95, source=df, fill_alpha=0.6, legend_field="metal",
           color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = dict(source=df, text_align="left", text_baseline="middle")

x = dodge("group", -0.4, range=p.x_range)

p.text(x=x, y="period", text="symbol", text_font_style="bold", **text_props)

p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number",
       text_font_size="11px", **text_props)

p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name",
       text_font_size="7px", **text_props)

p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass",
       text_font_size="7px", **text_props)

p.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"
p.hover.renderers = [r] # only hover element boxes

In [3]:
#
# output_notebook()  # works in a notebook but not on a Github Pages Jekyll site
# show(p)            # so we will use `components` instead

In [4]:
from bokeh.embed import components
from IPython.display import display, HTML

script, div = components(p)

html = f"""
<script src="https://cdn.bokeh.org/bokeh/release/bokeh-2.4.3.min.js"
        crossorigin="anonymous"></script>
<script src="https://cdn.bokeh.org/bokeh/release/bokeh-widgets-2.4.3.min.js"
        crossorigin="anonymous"></script>
{script}
{div}
"""

{% include info.html text="Why triple quotes?" %}

The `f""" """` is an f-string literal.{% fn 1 %}  They are great for programmatically constructing large chunks of non-Python code.

{{ '[PEP 498 &ndash; Literal String Interpolation](https://peps.python.org/pep-0498/)'  | fndetail: 1 }}

**Success!**

In [5]:
display(HTML(html))

{% include tip.html content="Hover over the elements to see additional information." %}

## Altair

Just works. *(though the browser console is complaining)*

In [6]:
import altair as alt

In [7]:
keep_columns = ['group', 'electronegativity', 'atomic number', 'atomic mass', 'symbol']
elements_ = elements.loc[elements.group != '-', keep_columns].copy()
elements_['group'] = elements_['group'].astype(int)

In [8]:
slider = alt.binding_range(min=1, max=18, step=1, name='group:')
selector = alt.selection_single(fields=['group'], bind=slider, init={'group': 17})

chart = (
    alt.Chart(elements_, title=f"Periodic electronegativity trends within groups").mark_point()
    .encode(
        x=alt.X('electronegativity:Q', scale=alt.Scale(domain=(0.5, 4.5))),
        y=alt.Y('symbol:N', sort=elements.symbol.to_numpy()),
        tooltip=['symbol','atomic number','atomic mass']
    )
    .add_selection(selector)
    .transform_filter(selector)
    .properties(width=400, height=200)
    .interactive()
)
chart

alt.Chart(...)